# Imports

In [1]:
import json
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import time
import utils

# Defining useful functions

In [9]:
def feasible(x, y, cap, incompatible_courses_array) :
    if  not feasible_1(x) :
        return False,1
    if not feasible_2(x, y, cap) :
        return False,2
    if not feasible_3(x, incompatible_courses_array) :
        return False,3
    return True,0

#This constraint states that every lecture should have a room
def feasible_1(x) :
    if (x.sum(axis=1) == np.ones(x.shape[0])).all() : 
        return True
    else :
        return False
    

#This constraint states that every lecture should be given in a room large enough 
def feasible_2(x, y, cap):
    return (np.dot(y,x)<=cap).all()

def feasible_3(x,incompatible_courses_array) :
    for i in range(x.shape[0]):
        j = np.where(x[i,:]==1)[0][0]
        inc_courses_at_j = x[incompatible_courses_array[i],j]
        if inc_courses_at_j.sum()>= 1 :
            print(i,j)
            return False
    return True 

def feasible_3_bis(x, day) :
    for j in range(x.shape[1]):
        lectures_indexes = np.where(x[:,j]==1)[0]
        lectures_names = [day.course_code_dict_index_to_name[k] for k in lectures_indexes]
        for name in lectures_names:
            inc_for_name = day.incompatible_courses_dict[name]
            for elem in lectures_names : #intersect is enough
                if elem in inc_for_name:
                    print(f"{elem} is part of the incompatible courses for {name}")
                    return False
    return True

def compatible_courses(list_of_course_names,day):
    for name in list_of_course_names : 
        inc_lectures = day.incompatible_courses_dict[name]
        for elem in list_of_course_names : 
            if elem in inc_lectures:
                return False
    return True

def valid_array(day_Information):
    for i in range(len(day_Information.incompatible_courses_array)):
        for elem in day_Information.incompatible_courses_array[i]:
            if i not in day_Information.incompatible_courses_array[elem] :
                print("Incompatible array is not symmetric")
                return False
    return True



def bad_dict(day_Information):
    for key in day_Information.incompatible_courses_dict.keys():
        if key in day_Information.incompatible_courses_dict[key]:
            print("Problem")
            return False
    return True

def bad_array(day_Information):
    for i in range(len(day_Information.incompatible_courses_array)):
        if i in day_Information.incompatible_courses_array[i]:
            print("Problem")
            return False
    return True

def compute_objective(x, day_Information, nb_of_students):
    obj = 0
    D = room.distance_matrix
    for s in range(nb_of_students):
        student_courses = day_Information.ordered_courses_for_student[s] 
        for i in range(len(student_courses)-1):
            obj += np.dot(np.dot(x[student_courses[i],:].T, D), x[student_courses[i+1],:])
    return obj

## Finding a feasible solution

In [3]:
import copy
import random

def generate_solution(nb_of_lectures, nb_of_rooms, day_Information, room, k):
    x = np.zeros((nb_of_lectures, nb_of_rooms))
    y = day_Information.nb_of_students_per_course
    y_with_name = day_Information.nb_of_students_per_course_code
    cap = room.capacities
    cap_with_name = room.capacity
    inc_courses_array = copy.deepcopy(day_Information.incompatible_courses_array)
    inc_courses_dict = copy.deepcopy(day_Information.incompatible_courses_dict)
    sorted_y_with_name = dict(sorted(y_with_name.items(), key=lambda item: item[1], reverse=True))
    sorted_cap_with_name = dict(sorted(cap_with_name.items(), key=lambda item: item[1], reverse=True))
    dynamic_y = copy.deepcopy(sorted_y_with_name)
    dynamic_cap = copy.deepcopy(sorted_cap_with_name)

    while bool(dynamic_cap) and bool(dynamic_y):
        room_name, room_cap = next(iter(dynamic_cap.items()))
        room_id = room.name_to_index[room_name]

        first_k_items = list(dynamic_y.items())[:k]
        course_name, course_stud = random.choice(first_k_items)
        #course_name, course_stud = next(iter(dynamic_y.items()))
        course_id = day_Information.course_code_dict_name_to_index[course_name]
        #print(course_name+" Out in room" + room_name)
        if course_stud <= room_cap:
            x[course_id,room_id] = 1
            del dynamic_y[course_name]

        else :
            #print(f"Room {room_name, room_cap} is too small for {course_name, course_stud} ")
            # raise ValueError
            return x, False
        
        #Now we try to put as many compatible lectures as possible in this room
        compatible_lectures = (set(dynamic_y.keys()) - inc_courses_dict[course_name]) -  {course_name}
        
        #We first convert this set to a dict {course_code : nb_of_students} where we only take good-sized lectures and sort
        compatible_lectures_d = {x: y_with_name[x] for x in compatible_lectures if y_with_name[x] <= room_cap}
        compatible_lectures_d = dict(sorted(compatible_lectures_d.items(), key=lambda item: item[1], reverse=True))
        
        while bool(compatible_lectures_d):
            #We take the first lecture 
            if course_name in compatible_lectures_d:
                print("Problem")
            next_course_name, next_course_stud = next(iter(compatible_lectures_d.items()))
            next_course_id = day_Information.course_code_dict_name_to_index[next_course_name]
            #print(next_course_name+" In in room "+room_name)
            if next_course_stud <= room_cap:
                x[next_course_id,room_id] = 1
                del dynamic_y[next_course_name]
                del compatible_lectures_d[next_course_name]
                #remove course from comp and dyn_y
            else : 
                #print(f"Room {room_name, room_cap} is too small for {next_course_name, next_course_stud} ")
                #raise ValueError
                break
            #Remove the course from compatible_lectures and remove his incompatible lectures
    #         del compatible_lectures_d[next_course_name]
            keys_to_remove = inc_courses_dict[next_course_name]
            keys_to_remove.add(next_course_name)
            compatible_lectures_d = { k : v for k,v in compatible_lectures_d.items() if k not in keys_to_remove} # This preserves order normally (?)

        compatible_lectures.clear()
        compatible_lectures_d.clear()
        
        del dynamic_cap[room_name]
    return x, True

# Solution solver

In [4]:
def solution_solver(N, k, nb_of_students ,nb_of_lectures, nb_of_rooms, day_Information, room):
    nb_of_feasible = 0
    nb_of_non_feasible = 0
    obj = []
    best_x = None
    best_obj = 2e20
    for i in range(N):
        x, status = generate_solution(nb_of_lectures, nb_of_rooms, day_Information, room, k)
        if feasible(x, day_Information.nb_of_students_per_course, room.capacities, day_Information.incompatible_courses_array)[0]  :
            nb_of_feasible += 1
            curr_obj = compute_objective(x, day_Information, nb_of_students)
            obj.append(curr_obj)
            if curr_obj < best_obj :
                best_obj = curr_obj
                best_x = x 
        else :
            nb_of_non_feasible += 1
    return best_x, best_obj, obj, nb_of_feasible, nb_of_non_feasible

# Local Search

In [5]:
def swap_2_opt(x, day_Information, room, nb_of_students, m):
    obj = []
    for j in range(x.shape[1]):
        if j%5 == 0:
            print(j)
        curr_room_cap = room.capacities[j]
        lec_indexes = np.where(x[:, j] == 1)[0]
        if len(lec_indexes) :
            largest_lec = day_Information.nb_of_students_per_course[lec_indexes].max()
        else :
            continue

        # Find all rooms that can accommodate the largest lecture in the current room
        target_rooms = np.where(room.capacities >= largest_lec)[0]
        # Only consider a subset of them of size m 
        target_rooms = np.random.choice(target_rooms, size=m, replace=False)
        # Iterate through feasible rooms and perform the swap
        for room_idx in target_rooms:
            lectures_in_targ = np.where(x[:,room_idx]==1)[0]
            if len(lectures_in_targ)>=1 :
                largest_lec_in_targ = day_Information.nb_of_students_per_course[lec_indexes].max()
            else :
                continue
                
            if room_idx != j and largest_lec_in_targ <= curr_room_cap:
                # Create a copy of the current solution
                x_new = x.copy()

                # Swap the room assignments between the current room and the target room
                x_new[lec_indexes, j] = 0
                x_new[lec_indexes, room_idx] = 1
                x_new[lectures_in_targ, room_idx] = 0
                x_new[lectures_in_targ,j] = 1
                
                if feasible(x_new, day_Information.nb_of_students_per_course, room.capacities, day_Information.incompatible_courses_array)[0]:
                    # Calculate the objective values for both the current and new solutions
                    objective_current = compute_objective(x, day_Information, nb_of_students)
                    objective_new = compute_objective(x_new, day_Information, nb_of_students)

                    # If the new solution has a better objective value, accept the swap
                    if objective_new < objective_current:
                        x = x_new
                        obj.append(objective_current)
                        print(f"Improved from {objective_current} to {objective_new}")
                        break

    return x


In [7]:
day_Information = utils.DayInformation('DataProject/DataProject/Events18.csv','DataProject/DataProject/students18_15.json')
day_Information.fill_matrix() # (nb_of_students, nb_of_lectures) matrix
day_Information.fill_df()
day_Information.fill_incompatible_dict()
day_Information.fill_ordered_courses_for_student()
room = utils.Room("DataProject/DataProject/SallesDeCours.csv", 'DataProject/DataProject/distances.xlsx', 1)
room.fill_matrix() # (nb_of_lectures, nb_of_lectures) matrix
enrollement = day_Information.matrix
nb_of_students = day_Information.matrix.shape[0]
nb_of_lectures = day_Information.matrix.shape[1]
nb_of_rooms = room.distance_matrix.shape[0]


In [10]:
import time
start = time.time()
best_x, best_obj, obj, nb_of_feasible, nb_of_non_feasible = solution_solver(100, 2, nb_of_students ,nb_of_lectures, nb_of_rooms, day_Information, room)
end = time.time()
print(f"It took {round(end-start,3)} s")
print(f"Nb of feasible : {nb_of_feasible}")
print(f"Nb of non-feasible : {nb_of_non_feasible}")

It took 36.45 s
Nb of feasible : 100
Nb of non-feasible : 0


In [11]:
obj = np.array(obj)
print(obj.mean())
print(obj.std())
print(obj.max())
print(obj.min())

91656.92
3408.402451824021
102435.0
82948.0


In [12]:
better_x = swap_2_opt(best_x, day_Information, room, nb_of_students,25)

0
Improved from 82948.0 to 82934.0
Improved from 82934.0 to 82892.0
5
10
Improved from 82892.0 to 82682.0
Improved from 82682.0 to 82575.0
Improved from 82575.0 to 82556.0
15
20
Improved from 82556.0 to 82546.0
Improved from 82546.0 to 82462.0
25
30
35
Improved from 82462.0 to 82266.0
Improved from 82266.0 to 81580.0
40
Improved from 81580.0 to 81440.0
45
Improved from 81440.0 to 81160.0
Improved from 81160.0 to 81092.0
50
55
Improved from 81092.0 to 81085.0
60
65
Improved from 81085.0 to 80902.0
Improved from 80902.0 to 80874.0
70
Improved from 80874.0 to 80238.0
Improved from 80238.0 to 80042.0
Improved from 80042.0 to 79901.0
Improved from 79901.0 to 79711.0
75
Improved from 79711.0 to 79627.0
80
Improved from 79627.0 to 78741.0
Improved from 78741.0 to 78559.0
85
Improved from 78559.0 to 78475.0
90
Improved from 78475.0 to 78315.0
95
Improved from 78315.0 to 77791.0
Improved from 77791.0 to 77453.0
100
Improved from 77453.0 to 77414.0
105
110
115
120
Improved from 77414.0 to 77411.